<a href="https://colab.research.google.com/github/ayoubbensakhria/finance_algo/blob/master/drug_interactions_and_toxicity_(analtox).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Install required packages and do necessary imports

---



In [ ]:
!pip install translators --upgrade

In [ ]:
from google.colab import drive
from lxml import html
from lxml.html.clean import clean_html, Cleaner
import pandas as pd
import numpy as np
import translators as ts
import json
import time
import random
drive.mount('/content/drive/')

Using United States server backend.


Mounted at /content/drive/


In [ ]:
# Posts random dates
start_date = "2022-12-30 1:30:10"
end_date = "2023-12-30 13:30:10"

_from=501
_to=1000

base_url = 'https://www.analyticaltoxicology.com/'

# authors count 
authors_count = 7

# products count
pdct_count = 4

# file name
filename = "/content/drive/My Drive/drugtox_export_{start}-{end}.csv".format(start=_from, end=_to)

In [ ]:
def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.
    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """
    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d %H:%M:%S', prop)

  

#2. Open CSVs


In [ ]:
drugs_df = pd.read_json('/content/drive/MyDrive/data/drugbank/_drugs.json')
interactions_df = pd.read_json('/content/drive/MyDrive/data/drugbank/_interactions.json')
synonyms_df = pd.read_json('/content/drive/MyDrive/data/drugbank/_synonyms.json')

In [ ]:
drugs_df = drugs_df.replace(r'^\s*$', np.nan, regex=True)

where = drugs_df['toxicity'].notnull()
# replace field that's entirely space (or empty) with NaN
filteredDrugs_df = drugs_df.loc[where]

len(filteredDrugs_df)

2010

In [ ]:
filteredDrugs_df = filteredDrugs_df.fillna('Not available.')

filteredDrugs_df['tags'] = filteredDrugs_df.apply(lambda row: (str(row['name'].lower()) + ', '+ str(row['synonyms']).lower()), axis =1)

filteredDrugs_df['processed'] = False

In [ ]:
filteredDrugs_df.head()

,CAS,Id,classification,clearance,description,halflife,indications,mechanism,name,structure,synonyms,toxicity,tags,processed
0,138068-37-8,0,<p><strong>Direct parent: </strong>Peptides</p...,* 164 ml/min [Healthy 18-60 yrs]\r\n* 139 ml/m...,Lepirudin is identical to natural hirudin exce...,Approximately 1.3 hours,For the treatment of heparin-induced thrombocy...,Lepirudin forms a stable non-covalent complex ...,Lepirudin,https://www.drugbank.ca/structures/DB00001/ima...,"Hirudin variant-1, Lepirudin recombinant","In case of overdose (eg, suggested by excessiv...","lepirudin, hirudin variant-1, lepirudin recomb...",False
1,205923-56-4,1,<p><strong>Direct parent: </strong>Peptides</p...,Female patients had 25% lower intrinsic cleara...,Cetuximab is an epidermal growth factor recept...,The mean half-life for Cetuximab is 114 hours ...,"Cetuximab, used in combination with irinotecan...",Cetuximab binds to the epidermal growth factor...,Cetuximab,https://www.drugbank.ca/structures/DB00002/ima...,"Cetuximab, Cétuximab, Cetuximabum, Immunoglobu...",Pulmonary Toxicity\r\n\r\nInterstitial lung di...,"cetuximab, cetuximab, cétuximab, cetuximabum, ...",False
2,143831-71-4,2,<p><strong>Direct parent: </strong>Peptides</p...,"Studies in rats indicate that, following aeros...",Dornase alfa is a biosynthetic form of human d...,Not available.,Used as adjunct therapy in the treatment of cy...,Dornase alfa is a biosynthetic form of human D...,Dornase alfa,https://www.drugbank.ca/structures/DB00003/ima...,Deoxyribonuclease (human clone 18-1 protein mo...,Adverse reactions occur at a frequency of < 1/...,"dornase alfa, deoxyribonuclease (human clone 1...",False
5,128270-60-0,5,<p><strong>Direct parent: </strong>Polypeptide...,* 3.4 mL/min/kg [Normal renal function]\r\n* 3...,Bivalirudin is a synthetic 20 residue peptide ...,* Normal renal function: 25 min (in normal con...,For treatment of heparin-induced thrombocytope...,Inhibits the action of thrombin by binding bot...,Bivalirudin,https://www.drugbank.ca/structures/DB00006/ima...,"Bivalirudina, Bivalirudinum, Hirulog","Based on a study by Gleason et al., the no-obs...","bivalirudin, bivalirudina, bivalirudinum, hirulog",False
6,53714-56-0,6,<p><strong>Direct parent: </strong>Peptides</p...,* 8.34 L/h [healthy male receiving a 1-mg IV b...,Leuprolide belongs to the general class of dru...,~3 hours,"For treatment of prostate cancer, endometriosi...",Leuprolide binds to the gonadotropin releasing...,Leuprolide,https://www.drugbank.ca/structures/DB00007/ima...,"(D-Leu(6),des-gly-NH2(10),pro-ethylamide(9))-g...",In rats subcutaneous administration of 250 to ...,"leuprolide, (d-leu(6),des-gly-nh2(10),pro-ethy...",False


In [ ]:
filteredDrugsTranslated_df = pd.DataFrame(columns = filteredDrugs_df.columns)
filteredDrugsTranslated_df = filteredDrugsTranslated_df[_from:_to]
filteredDrugs_df = filteredDrugs_df[_from:_to]

for index, row in filteredDrugs_df.iterrows():
  if row['processed'] == True:
    continue
  CAS = row['CAS']
  structure = row['structure']
  name = row['name']
  classification = row['classification']
  clearance = row['clearance']
  description = row['description'] 
  halflife = row['halflife']
  indications = row['indications']
  mechanism = row['mechanism']
  toxicity = row['toxicity']
  tags = row['tags']

  try:
    name = ts.google(name, from_language='en', to_language='fr')
    print(index, name)
    classification = ts.google(classification, from_language='en', to_language='fr')
    clearance = ts.google(clearance, from_language='en', to_language='fr')
    description = ts.google(description, from_language='en', to_language='fr')
    halflife = ts.google(halflife, from_language='en', to_language='fr')
    indications = ts.google(indications, from_language='en', to_language='fr')
    mechanism = ts.google(mechanism, from_language='en', to_language='fr')
    toxicity = ts.google(toxicity, from_language='en', to_language='fr')
    tags = ts.google(tags, from_language='en', to_language='fr')

    filteredDrugsTranslated_df = filteredDrugsTranslated_df.append({
      'CAS': CAS,
      'structure': structure,
      'name': name,
      'classification': classification,
      'clearance': clearance,
      'description': description,
      'halflife': halflife,
      'indications': indications,
      'mechanism': mechanism,
      'toxicity': toxicity,
      'tags': tags,
      }, ignore_index = True)
  
    filteredDrugs_df['processed'].loc[index] = True

  except:
    print('Error with: ', name)
    continue

In [ ]:
filteredDrugsTranslated_df.head()

,CAS,Id,classification,clearance,description,halflife,indications,mechanism,name,structure,synonyms,toxicity,tags,processed
0,32222-06-3,NaN,<p> <strong> Parent direct: </strong> vitamine...,"* 15,3 ml / h / kg [patients pédiatriques (tra...","Le calcitriol ou le 1,25-dihydroxycholécalcifé...",5-8 heures,Utilisé pour traiter une carence ou une insuff...,Le mécanisme d'action du calcitriol dans le tr...,Calcitriol,https://www.drugbank.ca/structures/DB00136/ima...,NaN,"Ld <band> 50 </ sub> (oral, rat) = 620 & mu; g...","calcitriol, (1s, 3r, 5z, 7e) -9,10-secocholest...",NaN
1,56-89-3,NaN,<p> <strong> parent direct: </strong> l-cystei...,Indisponible.,Un acide aminé dimère non essentiel lié par co...,Indisponible.,Il a été affirmé que la L-Cystéine possède des...,"Certaines conditions, par ex. Une surdose d'ac...",Cystine,https://www.drugbank.ca/structures/DB00138/ima...,NaN,"Avec des doses typiques de 1 à 1,5 grammes par...","Cystine, (R, R) -3,3'-dithiobis (acide 2-amino...",NaN
2,110-15-6,NaN,<p> <strong> parent direct: </strong> acides e...,Indisponible.,Un cristal incolore soluble dans l'eau avec un...,Indisponible.,"Pour la supplémentation nutritionnelle, égalem...",Le succinate est un composant essentiel du cyc...,Acide succinique,https://www.drugbank.ca/structures/DB00139/ima...,NaN,Rat oral ld <sub> 50 </ sub>: 2260 mg / kg,"acide succinique, acide 1,2-ététdicarboxylique...",NaN
3,7512-17-6,NaN,<p> <strong> parent direct: </strong> hexoses ...,Indisponible.,Le dérivé n-acétyl de la glucosamine.,Indisponible.,Pour le traitement et la prévention de l'arthr...,Le mécanisme d'action pour soulager la douleur...,N-acétylglucosamine,https://www.drugbank.ca/structures/DB00141/ima...,NaN,"La souris, Ld intraveineuse <sub> 50 </ sub> e...","N-acétylglucosamine, 2-acétamido-2-désoxy-d-gl...",NaN
4,56-86-0,NaN,<p> <strong> Parent direct: </strong> acide gl...,Indisponible.,Un peptide qui est un homopolymère d'acide glu...,Indisponible.,Considéré comme des «aliments cérébraux» de la...,Le glutamate active les récepteurs ionotropes ...,Acide glutamique,https://www.drugbank.ca/structures/DB00142/ima...,NaN,Le glutamate provoque des dommages neuronaux e...,"Acide glutamique, (2S) -2-aminopentanedioc aci...",NaN


In [ ]:
filteredDrugsTranslated_df['pub_date'] = filteredDrugsTranslated_df.apply(lambda row: random_date(start_date, end_date, random.random()), axis =1)

# author 
filteredDrugsTranslated_df['author'] = filteredDrugsTranslated_df.apply(lambda row: random.randint(1,authors_count), axis =1)

# classification
filteredDrugsTranslated_df['classification'] = filteredDrugsTranslated_df.apply(lambda row: row['classification'].title().replace('Strong', 'strong').replace('P>', 'p>'), axis =1)

# slugs
filteredDrugsTranslated_df['slug'] = filteredDrugsTranslated_df.apply(lambda row: '{cas}-{name}-fr'.format(cas=row['CAS'], name= row['name']).replace(' ', '-'), axis =1)

filteredDrugsTranslated_df.head()

,CAS,Id,classification,clearance,description,halflife,indications,mechanism,name,structure,synonyms,toxicity,tags,processed,pub_date,author,slug
0,32222-06-3,NaN,<p> <strong> Parent Direct: </strong> Vitamine...,"* 15,3 ml / h / kg [patients pédiatriques (tra...","Le calcitriol ou le 1,25-dihydroxycholécalcifé...",5-8 heures,Utilisé pour traiter une carence ou une insuff...,Le mécanisme d'action du calcitriol dans le tr...,Calcitriol,https://www.drugbank.ca/structures/DB00136/ima...,NaN,"Ld <band> 50 </ sub> (oral, rat) = 620 & mu; g...","calcitriol, (1s, 3r, 5z, 7e) -9,10-secocholest...",NaN,2022-11-04 06:47:03,7,32222-06-3-Calcitriol-fr
1,56-89-3,NaN,<p> <strong> Parent Direct: </strong> L-Cystei...,Indisponible.,Un acide aminé dimère non essentiel lié par co...,Indisponible.,Il a été affirmé que la L-Cystéine possède des...,"Certaines conditions, par ex. Une surdose d'ac...",Cystine,https://www.drugbank.ca/structures/DB00138/ima...,NaN,"Avec des doses typiques de 1 à 1,5 grammes par...","Cystine, (R, R) -3,3'-dithiobis (acide 2-amino...",NaN,2022-12-07 20:11:16,6,56-89-3-Cystine-fr
2,110-15-6,NaN,<p> <strong> Parent Direct: </strong> Acides E...,Indisponible.,Un cristal incolore soluble dans l'eau avec un...,Indisponible.,"Pour la supplémentation nutritionnelle, égalem...",Le succinate est un composant essentiel du cyc...,Acide succinique,https://www.drugbank.ca/structures/DB00139/ima...,NaN,Rat oral ld <sub> 50 </ sub>: 2260 mg / kg,"acide succinique, acide 1,2-ététdicarboxylique...",NaN,2022-08-29 13:25:32,6,110-15-6-Acide-succinique-fr
3,7512-17-6,NaN,<p> <strong> Parent Direct: </strong> Hexoses ...,Indisponible.,Le dérivé n-acétyl de la glucosamine.,Indisponible.,Pour le traitement et la prévention de l'arthr...,Le mécanisme d'action pour soulager la douleur...,N-acétylglucosamine,https://www.drugbank.ca/structures/DB00141/ima...,NaN,"La souris, Ld intraveineuse <sub> 50 </ sub> e...","N-acétylglucosamine, 2-acétamido-2-désoxy-d-gl...",NaN,2022-10-27 21:44:17,6,7512-17-6-N-acétylglucosamine-fr
4,56-86-0,NaN,<p> <strong> Parent Direct: </strong> Acide Gl...,Indisponible.,Un peptide qui est un homopolymère d'acide glu...,Indisponible.,Considéré comme des «aliments cérébraux» de la...,Le glutamate active les récepteurs ionotropes ...,Acide glutamique,https://www.drugbank.ca/structures/DB00142/ima...,NaN,Le glutamate provoque des dommages neuronaux e...,"Acide glutamique, (2S) -2-aminopentanedioc aci...",NaN,2022-08-27 19:10:34,6,56-86-0-Acide-glutamique-fr


In [ ]:
filteredDrugsTranslated_df.to_csv(filename, sep = ';')

In [ ]:
links = pd.DataFrame(columns=['keywords', 'url', 'slug'])

links['keywords'] = filteredDrugsTranslated_df['tags']

links['slug'] = filteredDrugsTranslated_df['slug']

links['url'] = links.apply(lambda row: base_url+row['slug'], axis =1)

links.to_csv(filename+'links.csv', sep = ';')